<a href="https://colab.research.google.com/github/mehrdadrashidian/CG-Ecosystem/blob/master/SRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install flask flask-ngrok moviepy pydub openai-whisper
!pip install flask-ngrok
!apt-get install -y ngrok

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package ngrok


In [5]:
!pip install flask pyngrok moviepy pydub openai-whisper


In [ ]:
# نصب کتابخانه‌ها

from flask import Flask, request, jsonify, send_file, render_template_string
from pyngrok import ngrok
import os
import whisper
from moviepy.editor import VideoFileClip

# تنظیمات اولیه Flask
app = Flask(__name__)

# مسیر ذخیره فایل‌ها
UPLOAD_FOLDER = "/content/uploads"
OUTPUT_FOLDER = "/content/outputs"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# استخراج صدا از ویدئو
def extract_audio(video_path, audio_path):
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)

# تبدیل صوت به متن
def transcribe_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["segments"]

# ایجاد فایل SRT
def generate_srt(segments, output_srt_path):
    def format_time(seconds):
        millis = int((seconds - int(seconds)) * 1000)
        seconds = int(seconds)
        minutes, seconds = divmod(seconds, 60)
        hours, minutes = divmod(minutes, 60)
        return f"{hours:02}:{minutes:02}:{seconds:02},{millis:03}"

    with open(output_srt_path, "w", encoding="utf-8") as srt_file:
        for idx, segment in enumerate(segments):
            start_time = segment["start"]
            end_time = segment["end"]
            text = segment["text"]
            srt_file.write(f"{idx + 1}\n")
            srt_file.write(f"{format_time(start_time)} --> {format_time(end_time)}\n")
            srt_file.write(f"{text}\n\n")

@app.route('/upload', methods=['POST'])
def upload_video():
    if 'file' not in request.files:
        return jsonify({"error": "No file part in the request"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No file selected"}), 400

    # ذخیره فایل آپلود شده
    video_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(video_path)

    # استخراج صدا و تبدیل به متن
    audio_path = os.path.join(OUTPUT_FOLDER, "output_audio.wav")
    extract_audio(video_path, audio_path)
    segments = transcribe_audio(audio_path)

    # ایجاد فایل SRT
    srt_path = os.path.join(OUTPUT_FOLDER, "output_subtitles.srt")
    generate_srt(segments, srt_path)

    return send_file(srt_path, as_attachment=True)

@app.route('/')
def index():
    html_template = """
    <!doctype html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Video to SRT Converter</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f9;
                color: #333;
                display: flex;
                flex-direction: column;
                align-items: center;
                justify-content: center;
                height: 100vh;
                margin: 0;
            }
            h1 {
                color: #555;
            }
            form {
                background: white;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
                text-align: center;
            }
            input[type="file"] {
                margin: 10px 0;
            }
            input[type="submit"] {
                background-color: #007BFF;
                color: white;
                border: none;
                padding: 10px 20px;
                border-radius: 5px;
                cursor: pointer;
            }
            input[type="submit"]:hover {
                background-color: #0056b3;
            }
        </style>
    </head>
    <body>
        <h1>Video to Subtitle Converter</h1>
        <form method="post" action="/upload" enctype="multipart/form-data">
            <input type="file" name="file" accept="video/*" required>
            <br>
            <input type="submit" value="Convert to SRT">
        </form>
    </body>
    </html>
    """
    return render_template_string(html_template)

# اجرای سرور Flask
if __name__ == "__main__":
    # افزودن توکن ngrok
    NGROK_AUTH_TOKEN = "2rBB8oZuHhqKW1cfZb4yV4ZKYtb_59fGYc2gwf6PQuPrDuuFE"
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # راه‌اندازی ngrok و دریافت لینک عمومی
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")

    app.run()


Public URL: NgrokTunnel: "https://9231-34-42-202-154.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Jan/2025 23:55:22] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2025 23:55:22] "GET /favicon.ico HTTP/1.1" 404 -


MoviePy - Writing audio in /content/outputs/output_audio.wav


MoviePy - Done.


  checkpoint = torch.load(fp, map_location=device)

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

INFO:werkzeug:127.0.0.1 - - [14/Jan/2025 23:57:10] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jan/2025 00:07:33] "GET / HTTP/1.1" 200 -
